In [10]:
import os
import json
from osgeo import gdal, ogr
import numpy as np
import re
from collections import defaultdict

# This script takes a polygon vector file (typically as square tiles) with fields that indicate rasters to combine, order, create, tile, and merge
# The input gpkg should have the following fields:
    # 'fid' (required, unique, int, autocreated in qgis): used to append to the name of the tiled output rasters
    # 'layers' (required, JSON(string)): which has a Key (to indicate the order of bands to stack) and Value (source of the raster to stack)
        # Key: enter the order of the rasters to be stacked (1,2,3,4...)(must be unique per feature) and raster source type (either 'path' or 'constant') with a camma to separate them
        # Key examples: '1,path' (stack the bands found at Value in the First band(s)) or '4,constant' (create a raster with a constant Value in the Forth band(s))
        # Value: for path enter the local path of the raster, for constant enter the value to be burned into pixels
        # Value Example: for path: '/Users/Name/Desktop/Image.tif', for constant: '0'
        # Full example of JSON object: {'1,constant': '-999', '2,path': '/Users/Name/Desktop/Image.tif'}
        # Tip: to update JSON object in qgis expression builder, use: map('1,path', /Users/Name/Desktop/Image.tif)
        # Tip: Ensuring tiling vector and images are in the same SRS for good tiling
        # Tip: Temp layer may not work to create JSON field in
    # 'group' (optional, string): all features with the same group value will be merged together and saved.
        # There will then be a 'groups' folder created in the output directory with the merged tiles, the grouped tiles will be named 'Group{groupValue}'
    # 'include' (optional, string); this field is used to only process tiles which START with 'yes' (can have notes after); if not present, nothing will be filtered

# Input parameters
gpkg_file = "/Users/kanoalindiwe/Desktop/grid.gpkg" 
output_dir = "/Users/kanoalindiwe/Desktop/output/"
output_basename = "Tile"
pixel_size = 0.5  # Pixel size should be divisible by tile size and raster extent.
noDataValue = -9999 # (Default: -9999)
align_pixels = True # (Default: True) True aligns picles with pixel whole number grid, false aligns pixels with top left edge of tiling vector

#------------- Start of script--------------
# Open the GeoPackage file
driver = ogr.GetDriverByName("GPKG")
vector_ds = driver.Open(gpkg_file, 0)
if vector_ds is None:
    print("Failed to open GPKG file")
    exit()

# Get the layer from the GeoPackage
layer = vector_ds.GetLayer()
countOfRastersCreated = 0

print (len(layer), 'tiles in layer')
srs = layer.GetSpatialRef()

# Loop through each feature (polygon) in the GeoPackage
def CreateTile():
    includeIsSet = hasattr(feature, 'include')
    if includeIsSet and (feature.include.startswith('yes') if feature.include is not None else False) or not includeIsSet:
        global countOfRastersCreated
        geom = feature.GetGeometryRef()
        minX, maxX, minY, maxY = geom.GetEnvelope()
    
        # Define output filename for the clipped and stacked raster
        feature_id = feature.GetFID()
        
        try:
            featureGroup = feature.GetField('group')
            featureGroup_prefix = f"Group{featureGroup}_" if featureGroup else ""
        except:
            featureGroup_prefix = ""
            
        output_raster = os.path.join(output_dir, f"{featureGroup_prefix}{output_basename}{feature_id}.tif")
    
        # Get the 'layers' field (JSON string with file paths to the TIFs)
        layers_field = feature.GetField("layers")
        layers = json.loads(layers_field)
        print(f"Input #{feature_id}: {layers}")
    
        # Calculate raster size based on extent and pixel size
        x_res = int((maxX - minX) / pixel_size)
        y_res = int((maxY - minY) / pixel_size)
        
        # Clip rasters for each layer and stack them
        clipped_rasters = []
        for key, value in layers.items():
            clipped_raster = f"/vsimem/clipped_{key}_{os.path.basename(value)}"
            if 'path' in key:
                gdal.Warp(
                    clipped_raster,
                    value, 
                    cutlineWKT=geom,
                    cutlineSRS=srs,
                    cropToCutline=True,
                    dstNodata=noDataValue,
                    xRes=pixel_size,
                    yRes=pixel_size,
                    outputBounds=(minX, minY, maxX, maxY),
                    resampleAlg="near",
                    outputType=gdal.GDT_Int16,
                    targetAlignedPixels=align_pixels
                )
            elif 'constant' in key:
                # Create the raster
                target_ds = gdal.GetDriverByName('GTiff').Create(
                    clipped_raster, x_res, y_res, 1, gdal.GDT_Int16)
                target_ds.SetProjection(layer.GetSpatialRef().ExportToWkt())
                target_ds.SetGeoTransform((minX, pixel_size, 0, maxY, 0, -pixel_size))
        
                target_ds.GetRasterBand(1).WriteArray(np.full((y_res, x_res), int(value), dtype=np.int16))
                target_ds.GetRasterBand(1).SetNoDataValue(noDataValue)
            
                # Cleanup
                target_ds = None
            else:
                print('Eror: invalid key raster source')
            clipped_rasters.append(clipped_raster)
            
    
        # Stack the clipped rasters into one output file
        vrt_options = gdal.BuildVRTOptions(separate=True)
        vrt_ds = gdal.BuildVRT("/vsimem/stacked.vrt", clipped_rasters, options=vrt_options)
        
        gdal.Translate(output_raster, vrt_ds, format="GTiff", noData=None)
    
        print(f"Output({vrt_ds.RasterCount} bands): {output_raster}")
        countOfRastersCreated = countOfRastersCreated + 1
    
        # Cleanup VRT
        vrt_ds = None
        gdal.Unlink(clipped_raster)
    else:
        pass

for feature in layer:
    CreateTile()


# Close the GeoPackage dataset
vector_ds = None
print(f'{countOfRastersCreated} rasters created')
print(len([f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f)) and output_basename in f]), 'tiles in output folder')

4 tiles in layer
Input #1: {'1,path': '/Users/kanoalindiwe/Desktop/Image.tif', '2,constant': '1'}
Output(9 bands): /Users/kanoalindiwe/Desktop/output/Grouptop_Tile1.tif
Input #2: {'1,path': '/Users/kanoalindiwe/Desktop/Image.tif', '2,constant': '1'}
Output(9 bands): /Users/kanoalindiwe/Desktop/output/Grouptop_Tile2.tif
Input #3: {'1,path': '/Users/kanoalindiwe/Desktop/Image.tif', '2,constant': '1'}
Output(9 bands): /Users/kanoalindiwe/Desktop/output/Groupbottom_Tile3.tif
Input #4: {'1,path': '/Users/kanoalindiwe/Desktop/Image.tif', '2,constant': '1'}
Output(9 bands): /Users/kanoalindiwe/Desktop/output/Groupbottom_Tile4.tif
4 rasters created
4 tiles in output folder


In [11]:
# Merge Rasters
def find_files(directory):
    # Dictionary to hold lists of files sorted by the group identifier
    grouped_files = defaultdict(list)

    # Walk through the directory
    for filename in os.listdir(directory):
        # Check if the filename starts with "Group"
        if filename.startswith("Group"):
            # Use regex to extract the part after "Group" and before "_"
            match = re.search(r'^Group(\w+)_', filename)
            if match:
                group_identifier = match.group(1)
                grouped_files[group_identifier].append(filename)

    # Convert the dictionary to a sorted list of arrays
    sorted_grouped_files = [grouped_files[key] for key in sorted(grouped_files.keys())]

    return sorted_grouped_files

def merge_groups(sorted_grouped_files, output_directory):
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Merge each group of rasters
    for group_files in sorted_grouped_files:
        if group_files:
            # Assuming the group identifier is derived from the first file
            group_name = os.path.basename(group_files[0]).split('_')[0]
            merged_raster_path = os.path.join(output_directory, f"{group_name}.tif")

            # Perform the merging using gdal.Warp
            print(f'Merging: {group_files}')
            gdal.Warp(merged_raster_path, [os.path.join(output_dir, f) for f in group_files], format='GTiff')

            print(f'Merged: {merged_raster_path}')

# Define output directory for groups
groups_output_dir = os.path.join(output_dir, 'groups')

# Find files
result = find_files(output_dir)

# Merge groups
merge_groups(result, groups_output_dir)

# Count files
countGroups = len([f for f in os.listdir(groups_output_dir) if os.path.isfile(os.path.join(groups_output_dir, f)) and 'Group' in f])
countTiles = len([f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f)) and output_basename in f])
print(f'{countTiles} tiles, {countGroups} groups (by count of files)')

Merging: ['Groupbottom_Tile3.tif', 'Groupbottom_Tile4.tif']
Merged: /Users/kanoalindiwe/Desktop/output/groups/Groupbottom.tif
Merging: ['Grouptop_Tile1.tif', 'Grouptop_Tile2.tif']
Merged: /Users/kanoalindiwe/Desktop/output/groups/Grouptop.tif
4 tiles, 2 groups (by count of files)
